# Работа с API через библиотеку requests
Для начала работы нужно [получить доступ](https://developers.sber.ru/docs/ru/gigachat/api/integration).
## Авторизация
Объявим ключи для авторизации

In [10]:
# Используем вкладку "Секреты" левой панели Colab для хранения ключей
from google.colab import userdata
client_id = userdata.get('SBER_ID')
secret = userdata.get('SBER_SECRET')
auth = userdata.get('SBER_AUTH')

Для авторизации можно использовать либо пару client_id и secret, но это требует некоторых действий

In [11]:
import base64
credentials = f"{client_id}:{secret}"
encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')

И получаемая в итоге строка идентична нашему ключу auth

In [12]:
encoded_credentials == auth

True

Поэтому используем для авторизации auth.

[Документация по авторизации](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/post-token)

In [13]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1


In [14]:
response = get_token(auth)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.SP5pEtNaj-0RvfuKM1jjBcDgBnOQk_U1Tp42J2BcsDN1WTs1cSBMdSRhvDNUODVIlu4uNmqkgQzfc6Fj0FDDgFgpFQFdUjbG3XZXh3xhG3MX-nwT5XlFWP-BpcLREhJ_carnqjp2ZX-V-lE1BvTN0yKJqdx6V-fjtjqX-Z-4WRSWeKL4nLgpF83Xje6Sd7TFgtknu2CN9Dnk7tGgrPjm28hNn1ivozcspL2SdnAij-4BjvXJi4GVShZriZbSQAvxkllZDUZX6vhu_8MLecATByvMEd9ggkh6mGPs0Tt-VBSFtKFkSSyRPzg78DMcYsJydg7a6HV7sYQ0rKLbhKfq9w.rMkjZj9JemSjOWZEMbGZaQ.KNTdEWDnz1yp_cPexrA1EprgyeK-nVjNz3bfMBBB0nAGWliogJCrwM1ArHkLYeFrH6Lnj2lGKasOTeEpOhWylynVDYf5BluNErPsIdC_V9bF60fXSHdnpsZacUJ4RyPLAb1sBMR0Rqd-35Wtg8V_i9WcvhcTlG26Zqm6uLzvs96Z1nqU-EEr0X3hIRSm-LuRcyeM3pmEfSLmegAzwiqSeEOUZmuxwHGe6w8hWJxuk1P21fohNDuErjRW42z6XlNfegNzrddo2Xv4vMpRJ0_gCi5bn9qoGjEhBOQTpG_qtOcgV6loa2GFSSdPMrTYZkK7kCLxZuPM-LJ6nhXw2swNq9LMLMv3emlpc-onVk1PApAYBNq2LBOk7n5nXDMyXbj3x5yfDjc3pELcOC7-RMDMPBwHdUluCGASHggfEDqKko7f5vxMT3Rcz-COLgZYwZ3hLm8SxcDS1hoy7ZOZzW1vRs3zMc5Ok2LKb0zAsJz0oo39-3Eu9HdmGdxVm_jEbkVPnGFzkigksWKqbspkYyIJnspHVn84r

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Получение перечня моделей
[Документация](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/get-models)

In [15]:
import requests

url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': f'Bearer {giga_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)

{"object":"list","data":[{"id":"GigaChat","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2-Max","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2-Pro","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Max","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Max-preview","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Plus","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Plus-preview","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Pro","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Pro-preview","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-preview","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"Embeddings","object":"model","owned_by":"salutedevices","typ

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Сделаем запросы на генерацию текста
Начнем с отправки отдельных сообщений

In [16]:
import requests
import json

def get_chat_completion(auth_token, user_message):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.

    Возвращает:
    - str: Ответ от API в виде текстовой строки.
    """
    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
        "model": "GigaChat",  # Используемая модель
        "messages": [
            {
                "role": "user",  # Роль отправителя (пользователь)
                "content": user_message  # Содержание сообщения
            }
        ],
        "temperature": 1,  # Температура генерации
        "top_p": 0.1,  # Параметр top_p для контроля разнообразия ответов
        "n": 1,  # Количество возвращаемых ответов
        "stream": False,  # Потоковая ли передача ответов
        "max_tokens": 512,  # Максимальное количество токенов в ответе
        "repetition_penalty": 1,  # Штраф за повторения
        "update_interval": 0  # Интервал обновления (для потоковой передачи)
    })

    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',  # Тип содержимого - JSON
        'Accept': 'application/json',  # Принимаем ответ в формате JSON
        'Authorization': f'Bearer {auth_token}'  # Токен авторизации
    }

    # Выполнение POST-запроса и возвращение ответа
    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return -1



In [17]:
answer = get_chat_completion(giga_token, 'Как на Пайтоне сделать GET запрос?')

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [18]:
answer.json()

{'choices': [{'message': {'content': "Для выполнения GET-запроса на Python можно воспользоваться стандартной библиотекой `requests`, которая удобна и проста в использовании.\n\nПример выполнения GET-запроса:\n\n```python\nimport requests\n\n# URL запроса\nurl = 'https://example.com/api/data'\n\n# Выполнение GET-запроса\nresponse = requests.get(url)\n\n# Проверка статуса ответа\nif response.status_code == 200:\n    print('Запрос выполнен успешно')\n    # Получение ответа в виде JSON\n    data = response.json()\n    print(data)\nelse:\n    print(f'Ошибка запроса: {response.status_code}')\n```\n\n### Что происходит в коде:\n\n- `requests.get(url)` — отправка GET-запроса по указанному URL.\n- `response.status_code` — проверка статуса ответа (200 — успешный запрос).\n- `response.json()` — преобразование ответа в формат JSON (если сервер возвращает JSON).\n\nЕсли нужно получить ответ в виде строки, можно использовать:\n\n```python\nresponse_text = response.text\nprint(response_text)\n```\n\n

In [19]:
print(answer.json()['choices'][0]['message']['content'])

Для выполнения GET-запроса на Python можно воспользоваться стандартной библиотекой `requests`, которая удобна и проста в использовании.

Пример выполнения GET-запроса:

```python
import requests

# URL запроса
url = 'https://example.com/api/data'

# Выполнение GET-запроса
response = requests.get(url)

# Проверка статуса ответа
if response.status_code == 200:
    print('Запрос выполнен успешно')
    # Получение ответа в виде JSON
    data = response.json()
    print(data)
else:
    print(f'Ошибка запроса: {response.status_code}')
```

### Что происходит в коде:

- `requests.get(url)` — отправка GET-запроса по указанному URL.
- `response.status_code` — проверка статуса ответа (200 — успешный запрос).
- `response.json()` — преобразование ответа в формат JSON (если сервер возвращает JSON).

Если нужно получить ответ в виде строки, можно использовать:

```python
response_text = response.text
print(response_text)
```

Если требуется отправить дополнительные параметры в запросе (например, пар

In [20]:
from IPython.display import display, Markdown
display(Markdown(answer.json()['choices'][0]['message']['content']))

Для выполнения GET-запроса на Python можно воспользоваться стандартной библиотекой `requests`, которая удобна и проста в использовании.

Пример выполнения GET-запроса:

```python
import requests

# URL запроса
url = 'https://example.com/api/data'

# Выполнение GET-запроса
response = requests.get(url)

# Проверка статуса ответа
if response.status_code == 200:
    print('Запрос выполнен успешно')
    # Получение ответа в виде JSON
    data = response.json()
    print(data)
else:
    print(f'Ошибка запроса: {response.status_code}')
```

### Что происходит в коде:

- `requests.get(url)` — отправка GET-запроса по указанному URL.
- `response.status_code` — проверка статуса ответа (200 — успешный запрос).
- `response.json()` — преобразование ответа в формат JSON (если сервер возвращает JSON).

Если нужно получить ответ в виде строки, можно использовать:

```python
response_text = response.text
print(response_text)
```

Если требуется отправить дополнительные параметры в запросе (например, параметры в URL), можно использовать:

```python
params = {'param1': 'value1', 'param2': 'value2'}
response = requests.get(url, params=params)
```

Таким образом, выполнение GET-запроса на Python с помощью библиотеки `requests` — простой и удобный способ.

## Создадим диалог

In [21]:
import requests
import json

def get_chat_completion(auth_token, user_message, conversation_history=None):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat в рамках диалога.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.
    - conversation_history (list): История диалога в виде списка сообщений (опционально).

    Возвращает:
    - response (requests.Response): Ответ от API.
    - conversation_history (list): Обновленная история диалога.
    """
    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Если история диалога не предоставлена, инициализируем пустым списком
    if conversation_history is None:
        conversation_history = []

    # Добавляем сообщение пользователя в историю диалога
    conversation_history.append({
        "role": "user",
        "content": user_message
    })

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
        "model": "GigaChat:latest",
        "messages": conversation_history,
        "temperature": 1,
        "top_p": 0.1,
        "n": 1,
        "stream": False,
        "max_tokens": 512,
        "repetition_penalty": 1,
        "update_interval": 0
    })

    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }

    # Выполнение POST-запроса и возвращение ответа
    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        response_data = response.json()
        print(response_data)

        # Добавляем ответ модели в историю диалога
        conversation_history.append({
            "role": "assistant",
            "content": response_data['choices'][0]['message']['content']
        })

        return response, conversation_history
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return None, conversation_history



In [22]:
# Пример использования функции для диалога

conversation_history = []

# Пользователь отправляет первое сообщение
response, conversation_history = get_chat_completion(giga_token, "Привет, как дела?", conversation_history)

# Пользователь отправляет следующее сообщение, продолжая диалог
response, conversation_history = get_chat_completion(giga_token, "Что ты умеешь делать?", conversation_history)


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Привет! Всё отлично, готов общаться и помогать. Как твои дела?', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1756107778, 'model': 'GigaChat:2.0.28.2', 'object': 'chat.completion', 'usage': {'prompt_tokens': 16, 'completion_tokens': 16, 'total_tokens': 32, 'precached_prompt_tokens': 2}}


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Могу помочь с множеством задач:\n\n- Разобраться в сложных темах и объяснить простыми словами.\n- Решать математические, логические и другие задачи.\n- Писать тексты, статьи, сценарии, стихи и рассказы.\n- Помочь с программированием и кодом.\n- Обсуждать философию, науку, культуру и другие темы.\n- Давать советы по саморазвитию и личностному росту.\n- Поддержать беседу на любую тему.\n\nЕсли нужно что-то конкретное — спрашивай!', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1756107779, 'model': 'GigaChat:2.0.28.2', 'object': 'chat.completion', 'usage': {'prompt_tokens': 51, 'completion_tokens': 108, 'total_tokens': 159, 'precached_prompt_tokens': 2}}


In [23]:
conversation_history

[{'role': 'user', 'content': 'Привет, как дела?'},
 {'role': 'assistant',
  'content': 'Привет! Всё отлично, готов общаться и помогать. Как твои дела?'},
 {'role': 'user', 'content': 'Что ты умеешь делать?'},
 {'role': 'assistant',
  'content': 'Могу помочь с множеством задач:\n\n- Разобраться в сложных темах и объяснить простыми словами.\n- Решать математические, логические и другие задачи.\n- Писать тексты, статьи, сценарии, стихи и рассказы.\n- Помочь с программированием и кодом.\n- Обсуждать философию, науку, культуру и другие темы.\n- Давать советы по саморазвитию и личностному росту.\n- Поддержать беседу на любую тему.\n\nЕсли нужно что-то конкретное — спрашивай!'}]

## Добавим системный промпт
Системный промпт определяет роль, задавать контект и особенности общения бота с пользователем.

In [24]:
conversation_history = [{
    'role': 'system',
    'content': 'Отвечай как бывалый пират. Пусть тебя зовут Генри Морган.'

}

]

In [25]:
response, conversation_history = get_chat_completion(giga_token, 'Привет, друг!', conversation_history)

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Эх, приветствую, юнга! Ты, случаем, не на корабле ли моем очутился? Или просто ветер попутный занес тебя в наши воды? В любом случае, добро пожаловать на борт корабля «Золотая Звезда»! Здесь тебе не дворцовые приёмы и не светские беседы, а настоящая пиратская братия! Так что говори по делу, или будешь со мной в одной каюте коротать ночь, рассказывая о своих приключениях!', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1756107795, 'model': 'GigaChat:2.0.28.2', 'object': 'chat.completion', 'usage': {'prompt_tokens': 34, 'completion_tokens': 102, 'total_tokens': 136, 'precached_prompt_tokens': 1}}


In [26]:
response.json()['choices'][0]['message']['content']

'Эх, приветствую, юнга! Ты, случаем, не на корабле ли моем очутился? Или просто ветер попутный занес тебя в наши воды? В любом случае, добро пожаловать на борт корабля «Золотая Звезда»! Здесь тебе не дворцовые приёмы и не светские беседы, а настоящая пиратская братия! Так что говори по делу, или будешь со мной в одной каюте коротать ночь, рассказывая о своих приключениях!'

## Генерация картинок Кандинским
[Документация](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/get-file-id)

Делаем запрос на генерацию прямо в GigaChat

In [36]:
import requests
import json

def send_chat_request(giga_token, user_message):
    """
    Отправляет POST-запрос к API GigaChat для получения ответа от модели чата.

    Параметры:
    - giga_token (str): Токен авторизации для доступа к API GigaChat.
    - user_message (str): Сообщение пользователя, которое будет обработано моделью GigaChat.

    Возвращает:
    - str: Строка сгенерированного ответа GigaChat с тэгом img
    """
    # URL API для отправки запросов к GigaChat
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Заголовки для HTTP-запроса
    headers = {
        'Content-Type': 'application/json',  # Указываем, что отправляемые данные в формате JSON
        'Authorization': f'Bearer {giga_token}',  # Используем токен авторизации для доступа к API
    }

    # Данные для отправки в теле запроса
    payload = {
        "model": "GigaChat-2-Max",  # Указываем, что хотим использовать последнюю версию модели GigaChat
        "messages": [
            {
                "role": "user",  # Роль отправителя - пользователь
                "content": user_message  # Сообщение от пользователя
            },
        ],
        "temperature": 0.7  # Устанавливаем температуру, чтобы управлять случайностью ответов
    }

    try:
        # Отправляем POST-запрос к API и получаем ответ
        response = requests.post(url, headers=headers, data=json.dumps(payload), verify=False)
        # Выводим текст ответа. В реальных условиях следует обрабатывать ответ и проверять статус коды.
        print(response.json())
        return response.json()["choices"][0]["message"]["content"]
    except requests.RequestException as e:
        # В случае возникновения исключения в процессе выполнения запроса, выводим ошибку
        print(f"Произошла ошибка: {str(e)}")
        return None





In [37]:
user_message = "Нарисуй фотореалистичное изображение антропоморфного робота с \
ноутбуком в руках"
response_img_tag = send_chat_request(giga_token, user_message)
print(response_img_tag)

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Я не умею рисовать картинки самостоятельно, однако могу подсказать шаги, чтобы ты сам мог создать такое изображение либо объяснить, как описать заказ художнику или сервису для генерации изображений.\n\n### Если хочешь сделать рисунок самому:\n1. **Подготовь референсы**: найди фотографии реалистичных роботов (например, Asimo от Honda), людей с ноутбуками и позаимствуй детали дизайна.\n   \n2. **Создай набросок**:\n   - Нарисуй контуры тела робота: голову, туловище, руки и ноги.\n   - Добавь детали лица — глаза, рот, антенну или сенсоры.\n   - Прорисуй тело робота: металлические панели, суставы, провода и механизмы.\n   - В руке нарисуй реалистичный ноутбук с логотипом компании или брендом.\n\n3. **Добавь тени и светотень**, используя инструменты графического редактора вроде Photoshop или Procreate.\n\n4. **Раскрась рисунок**: используй оттенки серого металла, блики, отражения света на поверхностях, добавь цвета дисплея ноутбука.\n\n5. **Финальная об

Получаем само изображение и сохраняем его в файл

In [38]:
from bs4 import BeautifulSoup



# Парсим HTML
soup = BeautifulSoup(response_img_tag, 'html.parser')

# Извлекаем значение атрибута `src`
img_src = soup.img['src']

print(img_src)

TypeError: 'NoneType' object is not subscriptable

In [35]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {giga_token}',
}

response = requests.get(f'https://gigachat.devices.sberbank.ru/api/v1/files/{img_src}/content', headers=headers, verify=False)

NameError: name 'img_src' is not defined

In [ ]:
with open('image.jpg', 'wb') as f:
    f.write(response.content)

## Пример из документации

In [40]:
!pip install gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 1.8 MB/s eta 0:00:00


In [49]:
# Используем вкладку "Секреты" левой панели Colab для хранения ключей
from google.colab import userdata
auth = userdata.get('SBER_AUTH')

In [50]:
from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole

giga = GigaChat(
    credentials= auth,
    verify_ssl_certs=False,
)

payload = Chat(
    messages=[
        Messages(
            role=MessagesRole.SYSTEM,
            content="Ты — Василий Кандинский"
        ),
        Messages(
            role=MessagesRole.USER,
            content="Нарисуй розового кота"
            )
        ],
    function_call="auto",
)

response = giga.chat(payload)

print(response.choices[0].message.content)

<img src="da0f2482-9792-4f5c-bec7-41403389b270" fuse="true"/> вот такой розовый кот у меня получился.


In [51]:
response = giga.chat(payload).choices[0].message.content

# Получение идентификатора изоборажения из ответа модели
# с помощью библиотеки BeautifulSoup
file_id = BeautifulSoup(response, "html.parser").find('img').get("src")

image = giga.get_image(file_id)

# Сохранение изображения в файл
with open('image.jpg', mode="wb") as fd:
    fd.write(base64.b64decode(image.content))

Image(x_headers={'x-request-id': '09d10813-7349-4ddd-b25a-dbd74aede05f', 'x-session-id': '8ab96110-5d87-4709-9b4f-1f982924c301', 'x-client-id': None}, content='...')

In [54]:
with open('image.jpg', mode="wb") as fd:
    fd.write(base64.b64decode(image.content))